In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import sklearn
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [3]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn import tree

1. Load data

In [4]:
with open('data.json') as f:
    data = json.load(f)
lst = json.loads(data[0])
df = pd.DataFrame(lst)

In [5]:
df.isnull().sum() #no missing data

days_since_first_order      0
days_since_last_order       0
is_newsletter_subscriber    0
orders                      0
items                       0
returns                     0
different_addresses         0
shipping_addresses          0
devices                     0
vouchers                    0
cc_payments                 0
paypal_payments             0
afterpay_payments           0
female_items                0
male_items                  0
unisex_items                0
wapp_items                  0
wftw_items                  0
mapp_items                  0
wacc_items                  0
macc_items                  0
mftw_items                  0
sprt_items                  0
msite_orders                0
desktop_orders              0
android_orders              0
ios_orders                  0
work_orders                 0
home_orders                 0
parcelpoint_orders          0
coupon_discount_applied     0
revenue                     0
customer_id                 0
dtype: int

In [6]:
#convert categorical feature to binary 
df['is_newsletter_subscriber'] = df['is_newsletter_subscriber'].map({"Y":1, "N":0})

2. Clean

In [7]:
df.describe()

,days_since_first_order,days_since_last_order,is_newsletter_subscriber,orders,items,returns,different_addresses,shipping_addresses,devices,vouchers,...,msite_orders,desktop_orders,android_orders,ios_orders,work_orders,home_orders,parcelpoint_orders,coupon_discount_applied,revenue,customer_id
count,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,...,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,191287.000000,1.912870e+05,191287.000000,1.912870e+05
mean,2325.415344,862.390832,0.661096,10.841887,24.544668,7.075745,1.258392,2.412438,1.645747,3.394993,...,2.732941,5.700529,0.227517,2.180901,1.203485,6.038685,0.101471,1.321704e+03,1515.276901,3.387436e+09
std,382.012579,832.876737,0.473339,19.191299,50.868840,25.923825,8.920020,4.493934,0.734939,7.428651,...,7.253788,11.975780,2.096664,8.206619,5.061593,12.875926,1.425184,3.905356e+04,3508.433432,2.094103e+08
min,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.559784e+02,-1.130000,3.016870e+09
25%,2271.000000,92.000000,0.000000,2.000000,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.800000e-03,140.840000,3.202564e+09
50%,2414.000000,544.000000,1.000000,4.000000,8.000000,1.000000,0.000000,2.000000,1.000000,1.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,5.520000e-02,443.100000,3.398475e+09
75%,2526.000000,1659.000000,1.000000,12.000000,25.000000,4.000000,0.000000,3.000000,2.000000,4.000000,...,3.000000,6.000000,0.000000,0.000000,0.000000,6.000000,0.000000,1.466000e-01,1477.225000,3.567305e+09
max,2813.000000,2329.000000,1.000000,1289.000000,3020.000000,2022.000000,1996.000000,1122.000000,4.000000,1135.000000,...,373.000000,1275.000000,244.000000,357.000000,223.000000,496.000000,290.000000,5.798000e+06,205655.020000,3.801194e+09


In [8]:
# swap coupon_discount_applied with revenue when revenue is equal to 0
m = df['revenue']==0
mp = {'coupon_discount_applied':'revenue','revenue':'coupon_discount_applied'}
df.update(df.loc[m].rename(mp,axis=1))

In [9]:
df['coupon_discount_applied'] = np.where(df.coupon_discount_applied>1,df.coupon_discount_applied/100,df.coupon_discount_applied)

In [10]:
# Remove error coupon_discount_applied : Remove 12 rows
df = df[(df['coupon_discount_applied']>=0)&(df['coupon_discount_applied']<1)]

In [11]:
df_copy = df.copy()

3. Engineer features

In [12]:
df['shopping_age'] = df['days_since_first_order'] - df['days_since_last_order']

df['average_items'] = df['items']/df['orders']
df['return_rate'] = df['returns']/df['orders']
df['different_addresses_rate'] = df['different_addresses']/df['orders']
df['shipping_addresses_rate'] = df['shipping_addresses']/df['orders']
df['devices_rate'] = df['devices']/df['orders']
df['vouchers_rate'] = df['vouchers']/df['orders']
df['cc_payment_rate'] = df['cc_payments']/df['orders']
df['paypal_payments_rate'] = df['paypal_payments']/df['orders']
df['afterpay_payments_rate'] = df['afterpay_payments']/df['orders']

df['female_items_rate'] = df['female_items']/df['items']
df['male_items_rate'] = df['male_items']/df['items']
df['unisex_items_rate'] = df['unisex_items']/df['items']

df['wapp_items_rate'] = df['wapp_items']/df['items']
df['wftw_items_rate'] = df['wftw_items']/df['items']
df['mapp_items_rate'] = df['mapp_items']/df['items']
df['mftw_items_rate'] = df['mftw_items']/df['items']
df['wacc_items_rate'] = df['wacc_items']/df['items']
df['macc_items_rate'] = df['macc_items']/df['items']
df['sprt_items_rate'] = df['sprt_items']/df['items']

df['msite_orders_rate'] = df['msite_orders']/df['orders']
df['desktop_orders_rate'] = df['desktop_orders']/df['orders']
df['android_orders_rate'] = df['android_orders']/df['orders']
df['ios_orders_rate'] = df['ios_orders']/df['orders']

df['work_orders_rate'] = df['work_orders']/df['orders']
df['home_orders_rate'] = df['home_orders']/df['orders']
df['parcelpoint_orders_rate'] = df['parcelpoint_orders']/df['orders']

df['aov'] = df['revenue']/df['orders']

In [13]:
df_engineer = df.iloc[:,33:]
df_asis = df[['customer_id','is_newsletter_subscriber','days_since_first_order', 'days_since_last_order', 'coupon_discount_applied']]
frames = [df_asis,df_engineer]
df_features = pd.concat(frames,axis=1,join='inner')

3.1 Scale feature

In [14]:
#keep categorical feature
df_keep = df_features[['customer_id','is_newsletter_subscriber']]

In [15]:
#rescale continous features
df_to_scale = df_features.iloc[:,2:]
scaler = StandardScaler()
col = df_to_scale.columns
df_to_scale = pd.DataFrame(scaler.fit_transform(df_to_scale))
df_to_scale.columns = col

In [16]:
df_engineered = pd.concat([df_keep,df_to_scale], axis=1, join='inner')

In [17]:
customer_id = df_engineered[['customer_id']]
x = df_engineered.drop(['customer_id'],axis=1)

4. Clustering

4.1. Select optimal number of cluster k

In [18]:
for num_clusters in range(2,7):
    clusterer = KMeans(n_clusters=num_clusters)
    preds = clusterer.fit_predict(x)
    centers = clusterer.cluster_centers_
    score = silhouette_score (x, preds, metric='euclidean')
    print ("For n_clusters = {}, Kmeans silhouette score is {})".format(num_clusters, score)) 

For n_clusters = 2, Kmeans silhouette score is 0.21125998709287902)
For n_clusters = 3, Kmeans silhouette score is 0.13733633778770388)
For n_clusters = 4, Kmeans silhouette score is 0.14502409314837464)
For n_clusters = 5, Kmeans silhouette score is 0.09269162766610581)
For n_clusters = 6, Kmeans silhouette score is 0.09865562682778083)


4.2. Cluster 2

In [19]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(x)
kmeans.cluster_centers_
clusters = kmeans.predict(x)

In [20]:
x["Cluster"] = clusters

5. Infer gender using Decision tree

In [21]:
X = x.iloc[:,:-1]
y = x.Cluster

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 
clf = DecisionTreeClassifier(criterion="entropy", max_depth=4)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)) #0.922428066017184

Accuracy: 0.9891597971383259


In [23]:
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_14 <= -0.63
|   |--- feature_14 <= -1.07
|   |   |--- feature_15 <= 0.68
|   |   |   |--- feature_18 <= -0.20
|   |   |   |   |--- class: 0
|   |   |   |--- feature_18 >  -0.20
|   |   |   |   |--- class: 0
|   |   |--- feature_15 >  0.68
|   |   |   |--- feature_14 <= -1.47
|   |   |   |   |--- class: 0
|   |   |   |--- feature_14 >  -1.47
|   |   |   |   |--- class: 0
|   |--- feature_14 >  -1.07
|   |   |--- feature_2 <= -0.09
|   |   |   |--- feature_15 <= 0.99
|   |   |   |   |--- class: 1
|   |   |   |--- feature_15 >  0.99
|   |   |   |   |--- class: 0
|   |   |--- feature_2 >  -0.09
|   |   |   |--- feature_15 <= 0.78
|   |   |   |   |--- class: 1
|   |   |   |--- feature_15 >  0.78
|   |   |   |   |--- class: 0
|--- feature_14 >  -0.63
|   |--- feature_15 <= 0.33
|   |   |--- feature_23 <= 10.72
|   |   |   |--- feature_22 <= 2.58
|   |   |   |   |--- class: 1
|   |   |   |--- feature_22 >  2.58
|   |   |   |   |--- class: 1
|   |   |--- feature_23 >  10.72
|   | 

In [24]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191263 entries, 0 to 191274
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   is_newsletter_subscriber  191263 non-null  float64
 1   days_since_first_order    191263 non-null  float64
 2   days_since_last_order     191263 non-null  float64
 3   coupon_discount_applied   191263 non-null  float64
 4   shopping_age              191263 non-null  float64
 5   average_items             191263 non-null  float64
 6   return_rate               191263 non-null  float64
 7   different_addresses_rate  191263 non-null  float64
 8   shipping_addresses_rate   191263 non-null  float64
 9   devices_rate              191263 non-null  float64
 10  vouchers_rate             191263 non-null  float64
 11  cc_payment_rate           191263 non-null  float64
 12  paypal_payments_rate      191263 non-null  float64
 13  afterpay_payments_rate    191263 non-null  f

In [31]:
x['female_flag'] = np.where(x.Cluster==1, 1,0)

5. Output

In [32]:
output = pd.concat([x,customer_id],axis=1, join='inner')
output = output[['customer_id','female_flag']]
output.to_csv('output.csv',index=False)